# Predictive text and text generation

By [Allison Parrish](http://www.decontextualize.com/)

This notebook is a whirlwind tour of how certain kinds of predictive text generation work! By "predictive text generation" what I mean is any text generation method that is based around a statistical model that, given a certain stretch of text, "predicts" which bit of text should come next, based on probabilities learned from an existing corpus of text.

The code is written in Python, but you don't really need to know Python in order to use the notebook. Everything's pre-written for you, so you can just execute the cells, making small changes to the code as needed.

## Working with text files

Before we get started, we'll first need some text! Grab two [plain text files from Project Gutenberg](http://www.gutenberg.org/) (or from another source of your choice) and save them to the same directory as this notebook. (I suggest working with two files because we'll be running some code explicitly to "compare" two texts. Also, I think seeing two different outputs from the text generation methods discussed in this notebook will help you better understand how those methods work.) The code in the following cell loads into Python variables the contents of *two plain text files*, assigned to variables `text_a` and `text_b`. You'll need to replace the filenames with the names of the files that you downloaded, keeping the quotation marks (`"`) intact.

In [1]:
text_a = open("1342-0.txt").read()
text_b = open("84-0.txt").read()

These variables are *strings*, which are essentially just long lists of the characters that occur in the text, in the order that they occur. The code in the following cell shows the first two hundred characters of text A:

In [2]:
print(text_a[:200])

﻿The Project Gutenberg EBook of Pride and Prejudice, by Jane Austen

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away 


You can change `text_a` to `text_b` to see the output from your second text, or change `200` to a number of your choosing.

The `random.sample()` function gives us a random sampling of the contents of a variable (as long as that variable is a sequence of things, like a string or a list). So, for example, to see twenty random characters from text B:

In [3]:
import random
random.sample(text_b, 20)

[' ',
 'm',
 'e',
 'y',
 't',
 ' ',
 's',
 'i',
 'e',
 'e',
 'e',
 'o',
 't',
 'e',
 ' ',
 ' ',
 'e',
 'a',
 't',
 'e']

This isn't incredibly helpful on its own, but you'll notice that the characters it drew (probably) more or less follow the expected letter distribution for English (i.e., lots of `e`s and `n`s and `t`s).

Perhaps more interesting would be to see a randomly-sampled list of *words*. To do this, we'll make separate variables for the words in the text, using a Python function called `.split()`, which takes a string and turns it into a list of words contained in that string. The following cell makes two new variables that contain the words from both texts respectively:

In [4]:
a_words = text_a.split()
b_words = text_b.split()

Now, ten random words from both text A and text B:

In [5]:
random.sample(a_words, 10)

['we',
 'arts',
 'the',
 'everything,',
 'but',
 'man-servant,',
 'that',
 'if',
 'you',
 'other']

In [6]:
random.sample(b_words, 10)

['“William,',
 'had',
 'but',
 'rose',
 'of',
 'the',
 'endowed',
 'up',
 'me,',
 'men;']

The code in the following cell uses Python's `Counter` object to count the *most common* letters in the first of these texts:

In [7]:
from collections import Counter
Counter(text_a).most_common(12)

[(' ', 113941),
 ('e', 70344),
 ('t', 47283),
 ('a', 42156),
 ('o', 41138),
 ('n', 38430),
 ('i', 36273),
 ('h', 33883),
 ('r', 33293),
 ('s', 33292),
 ('d', 22247),
 ('l', 21282)]

Specifying the `a_words` variable gives the most frequent *words* instead:

In [8]:
Counter(a_words).most_common(12)

[('the', 4205),
 ('to', 4121),
 ('of', 3662),
 ('and', 3309),
 ('a', 1945),
 ('her', 1858),
 ('in', 1813),
 ('was', 1795),
 ('I', 1740),
 ('that', 1419),
 ('not', 1356),
 ('she', 1306)]

Compare these to the most common words in text B:

In [9]:
Counter(b_words).most_common(12)

[('the', 4056),
 ('and', 2971),
 ('of', 2741),
 ('I', 2719),
 ('to', 2142),
 ('my', 1631),
 ('a', 1394),
 ('in', 1125),
 ('was', 993),
 ('that', 987),
 ('with', 700),
 ('had', 679)]

## Markov models: what comes next?

Now that we have the ability to find and record the n-grams in a text, it’s time to take our analysis one step further. The next question we’re going to try to answer is this: Given a particular n-gram in a text, what is most likely to come next?

We can imagine the kind of algorithm we’ll need to extract this information from the text. It will look very similar to the code to find n-grams above, but it will need to keep track not just of the n-grams but also a list of all units (word, character, whatever) that *follow* those n-grams.

Let’s do a quick example by hand. This is the same character-level order-2 n-gram analysis of the (very brief) text “condescendences” as above, but this time keeping track of all characters that follow each n-gram:

| n-grams |	next? |
| ------- | ----- |
|co| n|
|on| d|
|nd| e, e|
|de| s, n|
|es| c, (end of text)|
|sc| e|
|ce| n, s|
|en| d, c|
|nc| e|

From this table, we can determine that while the n-gram `co` is followed by n 100% of the time, and while the n-gram `on` is followed by `d` 100% of the time, the n-gram `de` is followed by `s` 50% of the time, and `n` the rest of the time. Likewise, the n-gram `es` is followed by `c` 50% of the time, and followed by the end of the text the other 50% of the time.

Exercise: Imagine (or even better, write out) what this table might look like if you were analyzing words instead of characters, with a source text of your choice.

### Markov chains: Generating text from a Markov model

The Markov models we created above don't just give us interesting statistical probabilities. It also allows us generate a *new* text with those probabilities by *chaining together predictions*. Here’s how we’ll do it, starting with the order 2 character-level Markov model of `condescendences`: (1) start with the initial n-gram (`co`)—those are the first two characters of our output. (2) Now, look at the last *n* characters of output, where *n* is the order of the n-grams in our table, and find those characters in the “n-grams” column. (3) Choose randomly among the possibilities in the corresponding “next” column, and append that letter to the output. (Sometimes, as with `co`, there’s only one possibility). (4) If you chose “end of text,” then the algorithm is over. Otherwise, repeat the process starting with (2). Here’s a record of the algorithm in action:

    co
    con
    cond
    conde
    conden
    condend
    condendes
    condendesc
    condendesce
    condendesces
    
As you can see, we’ve come up with a word that looks like the original word, and could even be passed off as a genuine English word (if you squint at it). From a statistical standpoint, the output of our algorithm is nearly indistinguishable from the input. This kind of algorithm—moving from one state to the next, according to a list of probabilities—is known as a Markov chain generator.

### Generating with Markovify

Fortunately, with the invention of digital computers, you don't have to perform this algorithm by hand! In fact, Markov chain text generation has been a pastime of poets and programmers going back [all the way to 1983](https://www.jstor.org/stable/24969024), so it should be no surprise that there are many implementations of the idea in Python that you can download and install. The one we're going to use is [Markovify](https://github.com/jsvine/markovify), a Markov chain text generation library originally developed for BuzzFeed, apparently. It comes with a lot of extra niceties that will make our lives easier, but underneath the hood, it implements an algorithm very similar to the one we just did by hand above.

To install Markovify on your computer, run the cell below:

In [14]:
!pip install markovify

You are using pip version 9.0.3, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


And then run this cell to make the library available in your notebook:

In [10]:
import markovify

The code in the following cell creates a new text generator, using the text in the variable specified to build the Markov model, which is then assigned to the variable `generator_a`.

In [11]:
generator_a = markovify.Text(text_a)

You can then call the `.make_sentence()` method to generate a sentence from the model:

In [12]:
print(generator_a.make_sentence())

But at last very rapidly, as well for the present, and, as they wished.


The `.make_short_sentence()` method allows you to specify a maximum length for the generated sentence:

In [13]:
print(generator_a.make_short_sentence(50))

Balls will be married!


By default, Markovify tries to generate a sentence that is significantly different from any existing sentence in the input text. As a consequence, sometimes the `.make_sentence()` or `.make_short_sentence()` methods will return `None`, which means that in ten tries it wasn't able to generate such a sentence. You can work around this by increasing the number of times it tries to generate a sufficiently unique sentence using the `tries` parameter:

In [14]:
print(generator_a.make_short_sentence(40, tries=100))

There were few people of fashion.”


Or by disabling the check altogether with `test_output=False`:

In [15]:
print(generator_a.make_short_sentence(40, test_output=False))

On Mr. and Mrs. Long.


### Changing the order

When you create the model, you can specify the order of the model using the `state_size` parameter. It defaults to 2. Let's make two model with different orders and compare:

In [16]:
gen_a_1 = markovify.Text(text_a, state_size=1)
gen_a_4 = markovify.Text(text_a, state_size=4)

In [17]:
print("order 1")
print(gen_a_1.make_sentence(test_output=False))
print()
print("order 4")
print(gen_a_4.make_sentence(test_output=False))

order 1
Not to utter more especially commercial redistribution.

order 4
However little Mr. Darcy might have done as well.”


In general, the higher the order, the more the sentences will seem "coherent" (i.e., more closely resembling the source text). Lower order models will produce more variation. Deciding on the order is usually a matter of taste and trial-and-error.

### Changing the level

Markovify, by default, works with *words* as the individual unit. It doesn't come out-of-the-box with support for character-level models. The following code defines a new kind of Markovify generator that implements character-level models. Execute it before continuing:

In [18]:
class SentencesByChar(markovify.Text):
    def word_split(self, sentence):
        return list(sentence)
    def word_join(self, words):
        return "".join(words)

Any of the parameters you passed to `markovify.Text` you can also pass to `SentencesByChar`. The `state_size` parameter still controls the order of the model, but now the n-grams are characters, not words.

The following cell implements a character-level Markov text generator for the word "condescendences":

In [19]:
con_model = SentencesByChar("condescendences", state_size=2)

Execute the cell below to see the output—it'll be a lot like what we implemented by hand earlier!

In [22]:
con_model.make_sentence()

'condendendescescendesces'

Of course, you can use a character-level model on any text of your choice. So, for example, the following cell creates a character-level order-7 Markov chain text generator from text A:

In [23]:
gen_a_char = SentencesByChar(text_a, state_size=7)

And the cell below prints out a random sentence from this generator. (The `.replace()` is to get rid of any newline characters in the output.)

In [26]:
print(gen_a_char.make_sentence(test_output=False).replace("\n", " "))

I could rest her eloping and took off my glove, and has since reached me two or three younger girls, too eager to offer to her head.


### Combining models

Markovify has a handy feature that allows you to *combine* models, creating a new model that draws on probabilities from both of the source models. You can use this to create hybrid output that mixes the style and content of two (or more!) different source texts. To do this, you need to create the models independently, and then call `.combine()` to combine them.

In [27]:
generator_a = markovify.Text(text_a)
generator_b = markovify.Text(text_b)
combo = markovify.combine([generator_a, generator_b], [0.5, 0.5])

The bit of code `[0.5, 0.5]` controls the "weights" of the models, i.e., how much to emphasize the probabilities of any model. You can change this to suit your tastes. (E.g., if you want mostly text A with but a *soupçon* of text B, you would write `[0.9, 0.1]`. Try it!) 

Then you can create sentences using the combined model:

In [30]:
print(combo.make_sentence())

We accordingly brought him from the observation of a little paler than usual, but more frequently sunk me into one of distinguishing the ladies both of whom we shall be the purport of this speech convinced my father said, with the former.


### Bringing it all together

I've pre-written some code below to make it easy for you to experiment and produce output from Markovify. Just make adjustments to the values assigned to the variables in the cell below:

In [31]:
# change to "word" for a word-level model
level = "char"
# controls the length of the n-gram
order = 7
# controls the number of lines to output
output_n = 14
# weights between the models; text A first, text B second.
# if you want to completely exclude one model, set its corresponding value to 0
weights = [0.5, 0.5]
# limit sentence output to this number of characters
length_limit = 280

(The lines beginning with `#` are "comments"—they don't do anything, they're just there to explain what's happening in the code.)

After making your changes above, run the cell below to generate text according to your parameters. Repeat as necessary until you get something you really like!

In [32]:
model_cls = markovify.Text if level == "word" else SentencesByChar
gen_a = model_cls(text_a, state_size=order)
gen_b = model_cls(text_b, state_size=order)
gen_combo = markovify.combine([gen_a, gen_b], weights)
for i in range(output_n):
    out = gen_combo.make_short_sentence(length_limit, test_output=False)
    out = out.replace("\n", " ")
    print(out)
    print()

Of neither from you paid the formerly beheld my endeavour to hear such a case.”

Jane was the rewarded and as Elizabeth longed to know the occasionally to her having events.

He has been commit my thoughts and thus traverse the pleasure and we returned in so dear to feel how unequalled.

Mr. Wickham immediately invited to the gratify whatever of honoured father would she had saved a human nature or pain was allayed by her complaints, and prepared forth in paragraph 1.F.3, a full discussed.

How can it be so unequal to the gaieties of activity in the fidgets.

What made her turn without some minutes, she read it like to give an accompany her uncle and altogether, soon went forgot.

“My fingers,” said to her sister's room, she soon joined by the wanted in my last look perfect and openly acknowledge is an advanced.

Yet he entertained by the inn.

He did not at all possible, white steeple and elevating her judgment for free      electronic work be imperfect in society of them, would be ma

What we're comparing here is called *unigram frequency*. ("Unigram" means a sequence of length one—more on this below.) For most English texts, the most frequent words in any given text will correspond closely to the most common 

## Neural network text prediction with `textgenrnn`

Like a [Markov chain](ngrams-and-markov-chains.ipynb), a recurrent neural network (RNN) is a way to make predictions about what will come next in a sequence. For our purposes, the sequence in question is a sequence of characters, and the prediction we want to make is *which character will come next*. Both Markov models and recurrent neural networks do this by using statistical properties of text to make a *probability distribution* for what character will come next, given some information about what comes before. The two procedures work very differently internally, and we're not going to go into the gory details about implementation here. (But if you're interested in the gory details, [here's a good place to start](https://karpathy.github.io/2015/05/21/rnn-effectiveness/).) For our purposes, the main *functional* difference between a Markov chain and a recurrent neural network is the *portion* of the sequence used to make the prediction. A Markov model uses a fixed window of history from the sequence, while an RNN (theoretically) uses the *entire history* of the sequence.

## Start with Markov

To illustrate, here's that Markov model of the word "condescendences." In a Markov model based on bigrams from this string of characters, you'd make a list of bigrams and the characters that follow those bigrams, like so:

| n-grams |	next? |
| ------- | ----- |
| co      | n |
| on      | d |
| nd      | e, e |
| de      | s, n |
| es      | c, (end of text) |
| sc      | e |
| ce      | n, s |
| en      | d, c |
| nc      | e |

You could also write this as a probability distribution, with one column for each bigram. The value in each cell indicates the probability that the character following the bigram in a given row will be followed by the character in a given column:

| n-grams | c | o | n | d | e | s | END |
| ------- | - | - | - | - | - | - | --- |
| co      | 0 | 0 | 1.0 | 0 | 0 | 0 | 0 | 
| on      | 0 | 0 | 0 | 1.0 | 0 | 0 | 0 | 
| nd      | 0 | 0 | 0 | 0 | 1.0 | 0 | 0 | 
| de      | 0 | 0 | 0.5 | 0 | 0 | 0.5 | 0 |
| es      | 0.5 | 0 | 0 | 0 | 0 | 0 | 0.5 |
| sc      | 0 | 0 | 0 | 0 | 1.0 | 0 | 0 |
| ce      | 0 | 0 | 0.5 | 0 | 0 | 0.5 | 0 |
| en      | 0.5 | 0 | 0 | 0.5 | 0 | 0 | 0 |
| nc      | 0 | 0 | 0 | 0 | 1.0 | 0 | 0 |

Each row of this table is a *probability distribution*, meaning that it shows how probable a given letter is to follow the n-gram in the original text. In a probability distribution, all of the values add up to 1.

Fitting a Markov model to the data is a matter of looking at each sequence of characters in a given text, and updating the table of probability distributions accordingly. To make a prediction from this table, you can "sample" from the probability distribution for a given n-gram (i.e., sampling from the distribution for the bigram `de`, you'd have a 50% chance of picking `n` and a 50% chance of picking `s`).

Another way of thinking about this Markov model is as a (hypothetical!) function *f* that takes a bigram as a parameter and returns a probability distribution for that bigram:

    f("ce") → [0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0]
    
(Note that the values at each index in this distribution line up with the columns in the table above.)
    
The items in the list returned from this function correspond to the probability for the corresponding next character, as given in the table. To sample from this list, you'd pick randomly among the indices according to their probabilities, and then look up the corresponding character by its position in the table.

To generate new text from this model:

1. Set your output string to a randomly selected n-gram
2. Sample a letter from the probability distribution associated with the n-gram at the end of the output string
3. Append the sampled letter to the end of the string
4. Repeat from (2) until the END token is reached

Of course, you don't write this function by hand! When you're creating a Markov model from your data (or "training" the model), you're essentially asking the computer to write this function *for you*. In this sense, a Markov model is a very simple kind of machine learning, since the computer "learns" the probability distribution from the data that you feed it.

## A (very) simplified explanation of RNNs

The mechanism by which a recurrent neural network "learns" probability distributions from sequences is much more sophisticated than the mechanism used in a Markov model, but functionally they're very similar: you give the computer some data to "train" on, and then ask it to automatically create a function that will return a probability distribution of what comes next, given some input. An RNN differs from a Markov chain in that to predict the next item in the sequence, you pass in *the entire sequence* instead of just the most recent n-gram.

In other words, you can (again, hypothetically) think of an RNN as a way of automatically creating a function *f* that takes a sequence of characters of arbitrary length and returns a probability distribution for which character comes next in the sequence. Unlike a Markov chain, it's possible to *improve* the accuracy of the probability distribution returned from this function by training on the same data multiple times.

Let's say that we want to train the RNN on the string "condescendences" to learn this function, and we want to make a prediction about what character is most likely to follow the sequence of characters "condescendence". When training a neural network, the process of learning a function like this works iteratively: you start off with a function that essentially gives a uniform probability distribution for each outcome (i.e., no one outcome is considered more likely than any other):

    f("condescendences") → [0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14] (after zero passes through the data)
    
... and as you iterate over the training data (in this case, the word "condescendences"), the probability distribution  gradually improves, ideally until it comes to accurately reflect the actual observed distribution (in the parlance, until it "converges"). After some number of passes through the data, you might expect the automatically-learned function to return distributions like this:

    f("condescendences") → [0.01, 0.02, 0.01, 0.03, 0.01, 0.9, 0.02] (after n passes through the data)

A single pass through the training data is called an "epoch." When it comes to any neural network, and RNNs in particular, more epochs is almost always better.

To generate text from this model:

1. Initialize your output string to an empty string, or a random character, or a starting "prefix" that you specify;
2. Sample the next letter from the distribution returned for the current output string;
3. Append that character to the end of the output string;
4. Repeat from (2)

Of course, in a real life application of both a Markov model and an RNN, you'd normally have more than seven items in the probability distribution! In fact, you'd have one element in the probability distribution for every possible character that occurs in the text. (Meaning that if there were 100 unique characters found in the text, the probability distribution would have 100 items in it.)

## Markov chains vs RNNs    

The primary benefit of an RNN over a Markov model for text generation is that an RNN takes into account *the entire history* of a sequence when generating the next character. This means that, for example, an RNN can theoretically learn how to close quotes and parentheses, which a Markov chain will never be able to reliably do (at least for pairs of quotes and parentheses longer than the n-gram of the Markov chain).

The drawback of RNNs is that they are *computationally expensive*, from both a processing and memory perspective. This is (again) a simplification, but internally, RNNs work by "squishing" information about the training data down into large matrices, and make predictions by performing calculations on these large matrices. That means that you need a lot of CPU and RAM to train an RNN, and the resulting models (when stored to disk) can be very large. Training an RNN also (usually) takes a lot of time.

Another consideration is the size of your corpus. Markov models will give interesting and useful results even for very small datasets, but RNNs require large amounts of data to train—the more data the better.

So what do you do if you *don't* have a very large corpus? Or if you don't have a lot of time to train on your corpus?

## RNN generation from pre-trained models

Fortunately for us, developer and data scientist [Max Woolf](https://github.com/minimaxir) has made a Python library called [textgenrnn](https://github.com/minimaxir/textgenrnn) that makes it really easy to experiment with RNN text generation. This library includes a model (according to the documentation) "trained on hundreds of thousands of text documents, from Reddit submissions (via BigQuery) and Facebook Pages (via my Facebook Page Post Scraper), from a very diverse variety of subreddits/Pages," and allows you to use this model as a starting point for your own training.

First install textgenrnn with `pip`. (You don't need to do this if you're running the notebook on Binder.)

In [32]:
!pip install --upgrade textgenrnn

Requirement already up-to-date: textgenrnn in /Users/allison/anaconda/lib/python3.6/site-packages
Requirement already up-to-date: h5py in /Users/allison/anaconda/lib/python3.6/site-packages (from textgenrnn)
Requirement already up-to-date: keras>=2.1.5 in /Users/allison/anaconda/lib/python3.6/site-packages (from textgenrnn)
Requirement already up-to-date: scikit-learn in /Users/allison/anaconda/lib/python3.6/site-packages (from textgenrnn)
Requirement already up-to-date: six in /Users/allison/anaconda/lib/python3.6/site-packages (from h5py->textgenrnn)
Requirement already up-to-date: numpy>=1.7 in /Users/allison/anaconda/lib/python3.6/site-packages (from h5py->textgenrnn)
Requirement already up-to-date: keras-applications>=1.0.6 in /Users/allison/anaconda/lib/python3.6/site-packages (from keras>=2.1.5->textgenrnn)
Requirement already up-to-date: scipy>=0.14 in /Users/allison/anaconda/lib/python3.6/site-packages (from keras>=2.1.5->textgenrnn)
Requirement already up-to-date: keras-prepr

Once it's installed, import the `textgenrnn` class from the package:

In [33]:
from textgenrnn import textgenrnn

Using TensorFlow backend.


And create a new `textgenrnn` object like so. (The `name` parameter controls the filename used when automatically saving the model to disk, so pick something descriptive!)

In [34]:
textgen = textgenrnn(name="text_a")

This object has a `.generate()` method which will, by default, generate text from the pre-trained model only.

In [37]:
textgen.generate()

A stream in his boys and they came out or something but any time whenever you have any street start in the start?



The `textgenrnn` library needs a data structure called a "list of strings" as its source text for training. We'll use Markovify's `split_into_sentences` method to turn our plain-text input files into lists of sentences like so:

In [38]:
from markovify.splitters import split_into_sentences
text_a_sentences = split_into_sentences(text_a)
text_b_sentences = split_into_sentences(text_b)

Here are five random sentences from both texts:

In [39]:
random.sample(text_a_sentences, 5)

['“But I was embarrassed.”',
 'She does not yet leave her dressing-room.',
 'This event had at last been despaired of, but it was then\ntoo late to be saving.',
 '“I am by no means of the opinion, I assure you,” said he, “that a ball\nof this kind, given by a young man of character, to respectable people,\ncan have any evil tendency; and I am so far from objecting to dancing\nmyself, that I shall hope to be honoured with the hands of all my fair\ncousins in the course of the evening; and I take this opportunity of\nsoliciting yours, Miss Elizabeth, for the two first dances especially,\na preference which I trust my cousin Jane will attribute to the right\ncause, and not to any disrespect for her.”',
 'I wonder what he can be doing there.”']

In [40]:
random.sample(text_b_sentences, 5)

['The light of that conflagration\nwill fade away; my ashes will be swept into the sea by the winds.',
 'Years will pass, and\nyou will have visitings of despair and yet be tortured by hope.',
 'My hand was already on the\nlock of the door before I recollected myself.',
 'Ever since the fatal night, the end of my labours, and the\nbeginning of my misfortunes, I had conceived a violent antipathy even\nto the name of natural philosophy.',
 'The\ndissecting room and the slaughter-house furnished many of my materials;\nand often did my human nature turn with loathing from my occupation,\nwhilst, still urged on by an eagerness which perpetually increased, I\nbrought my work near to a conclusion.']

To train a text generator on your own text, use the `.train_on_texts()` method, passing in a list of strings. The `num_epochs` parameter allows you to indicate how many epochs (i.e., passes over the data) should be performed. The more epochs the better, especially for shorter texts, but you'll get okay results even with just a few.

Training a neural network usually takes a really long time! So it makes sense to "try out" a text before committing to the many hours it might take to train the network on the full text. The following example trains the neural network on just the first 100 lines from text A, which lets you get an idea of what the output will look like when training on its entire contents. You'll notice that the `train_on_texts()` function prints output as it goes, showing what the generated text is likely to look like.

In [41]:
textgen.train_on_texts(text_a_sentences[:100], num_epochs=3)

Training on 8,345 character sequences.
Epoch 1/3
65/65 [==============================] - 28s 433ms/step - loss: 2.0301
####################
Temperature: 0.2
####################
“What the share is a single thing to have his that the share of the menthoo my dear.

“What is the suphed to the share of the sisters.”

“I have no used to he have no one of the share to he want to she she she she she want to she she has some of the wife is a silly.”

####################
Temperature: 0.5
####################
“What and sayche that you heard than the end of the sission so invision to you have you say the for the sissive with that and the amutagies it and so that the execrapation to have said has no only have so her than the morning.”

“I was some of the medichion what you will to have has onlation me in my dear.

“Prepe has he desig want to see the sandon today only have no replied the wife to deserve you send to not the dear that he deserve his the thing me.

####################
Temperature: 

After training, you can generate new text using the `.generate()` method again:

In [45]:
textgen.generate()

“I have a single play for the suite of the she will _l_.



The results aren't very interesting because by default the generator is very conservative in how it samples from the probability distribution. You can use the `temperature` parameter to make the sampling a bit more likely to pick improbable outcomes. The higher the value, the weirder the results. The default is 0.2, and going above 1.0 is likely to produce unacceptably strange results:

In [46]:
textgen.generate(temperature=0.5)

“I have no one of the thing and some of Mrs.



In [47]:
textgen.generate(temperature=0.9)

But it was finland end this a girls; it will be is deal his how much to informed her employed ahmedy it?”



In [51]:
textgen.generate(temperature=1.5)

RiteMTHr since 22,senses: Lengun of R,nowcets, fone therliefiу?



If you pass a number `n` to the `.generate()` method as its first parameter, `.generate()` will print out `n` instances of text generation from the model. The code in the following cell prints out ten examples from the specified temperature:

In [52]:
textgen.generate(10, temperature=0.5)

 10%|█         | 1/10 [00:03<00:28,  3.19s/it]

“I am sure to have a good termary in you when married use of the her intervice of the silly to make the surround of the fortune of the more of the surforthlowless will be must enough to have surved him to the meaning of the way to heard them?”



 20%|██        | 2/10 [00:04<00:22,  2.77s/it]

“My encode of the she difficult into the rest as a guybou and information in them in the formand is a she indenem.”



 30%|███       | 3/10 [00:06<00:16,  2.42s/it]

...

“When a posstee of Mrs Lizzy is for the formand introduction, that he was so this will tell you he came it on the man of AustenPromoda and Mr. Bennet and chance, at some of the she dear, that you he was a way to be seen so much to come on her in the more of think to be such a formand competion scol



 90%|█████████ | 9/10 [00:26<00:03,  3.42s/it]

“I have not she want to have a visit of this she will dear and many he cannot depresive of my little waited with the half on the wife who he was thinking to come on the her to see any she was to be a shirt and waited the sureself in the girls now.



100%|██████████| 10/10 [00:28<00:00,  3.10s/it]

“I have a was thereforretic the herself in the forse in a handsome of his a way that you know with you may sure it intended themselves now.



(This may take a little while.)

If you specify `return_as_list=True`, the `.generate()` method returns the results as a list instead of printing them out:

In [60]:
generated_strs = textgen.generate(10, temperature=0.5, return_as_list=True)
print(generated_strs)

100%|██████████| 10/10 [00:16<00:00,  1.59s/it]

['“When the such in one was not must sure it.”', 'I was so the streament in a settling it intended; that you must was that he wished; that he can he was a news of Mr. Bennet may so you will be any cannabu when can north them, and not make that you must know what we decimie and it is see it and it is any keve_ it any know themselves you amand in th', '“Does not that he was married them, and advice it in the her song.', '“I have a single tim for him to her and it taken his dear do you must think of the considers.”', '“But you have anything .', '“I hope a man of so finte them all of my dear, what you think of the said that I have to know what you the house of introduce of them so you you must make an alway and her wife.”', '“I have a balture of the his not depended with that I am sure if searched and therefore is a father.', 'A was theother and it is addrick them and anyone any keve_ her more of the married than the hand of herself so went in the hand to make to get ney with himmyyones ar

When you're satisfied with the results and you're ready to train on all of the sentences, just remove the `[:100]` from the call to `.train_on_texts()`:

In [ ]:
textgen.train_on_texts(text_a_sentences, num_epochs=5)

The textgenrnn library automatically saves the model to disk after each epoch in the same directory as this notebook. You can load a model you've previously trained by passing its filename to the `textgenrnn` function:

In [53]:
textgen = textgenrnn("text_a_weights.hdf5")

And then you can call the `.generate()` method as normal:

In [54]:
textgen.generate(10, temperature=0.5)

 10%|█         | 1/10 [00:03<00:27,  3.03s/it]

How make that see the such a said my best of the sister will be in a single will never to be the thing than them as you have a good term of they will be caperad in the for them, sick and her she been with Mrs.



 20%|██        | 2/10 [00:03<00:18,  2.30s/it]

“I have a wife are consider you may be abluse?”



 30%|███       | 3/10 [00:07<00:19,  2.72s/it]

...

“I have no used on the first of the sued of the sisters the wife had no also be replied that he is not must have a gue of the heaven to take the prepast on the amuse of the serios to find that it is to the heavy includedwiffling to she she designg the such she is no married; I was prepeyd and is a 



 90%|█████████ | 9/10 [00:20<00:02,  2.22s/it]

“Are you may was tour to be so muchdeliess them.”



100%|██████████| 10/10 [00:21<00:00,  1.94s/it]

“I said that I was so devereed to heard or so, that he can be one of them.”



(*Note*: If you're running this on Binder, make sure to download the weights from the notebook server's Home page! You can upload them again when you start a new session. Binder will automatically delete the data associated with inactive notebooks.)

### Generating with shorter texts

I've found that `textgenrnn` works especially well with very short, word-length texts. For example, download [this file of human moods](https://github.com/dariusk/corpora/blob/master/data/humans/moods.json) from Corpora Project, and put it in the same directory as this notebook.

Then load the JSON file and grab just the list of words naming moods:

In [56]:
import json
mood_data = json.loads(open("./moods.json").read())
moods = mood_data['moods']

And create another textgenrnn object:

In [57]:
mood_gen = textgenrnn(name="moods")

Now, train the RNN on these moods. One epoch will do the trick:

In [58]:
mood_gen.train_on_texts(moods, num_epochs=1)

Training on 6,651 character sequences.
Epoch 1/1
51/51 [==============================] - 23s 442ms/step - loss: 2.2959
####################
Temperature: 0.2
####################
art

derressed

instanted

####################
Temperature: 0.5
####################
inactly

alliee

tarreated

####################
Temperature: 1.0
####################
deed

tymed

joygucture



Now generate a list of new moods:

In [59]:
mood_gen.generate(25, temperature=0.5)

  8%|▊         | 2/25 [00:00<00:02,  9.93it/s]

owned

innepsess



 16%|█▌        | 4/25 [00:00<00:02,  8.62it/s]

insterette

resepted



 24%|██▍       | 6/25 [00:00<00:02,  7.49it/s]

...

distanted

exploured



 92%|█████████▏| 23/25 [00:02<00:00,  6.53it/s]

feated

belsiffful



100%|██████████| 25/25 [00:03<00:00,  7.31it/s]

bad

volutic



## Further reading

* [This notebook from the creator of textgenrnn](https://github.com/minimaxir/textgenrnn/blob/master/docs/textgenrnn-demo.ipynb) covers everything about the library that I covered in this tutorial—and much more, including how to start generation from a particular "seed" and how to save and load models (useful if you spent an afternoon training a model on your own corpus and don't want to have to do it again!)
* Take a look at [Janelle Shane's wonderful overview of how she uses RNNs in her process](http://aiweirdness.com/faq). And then take a look at her [wonderful creative work with RNNs](http://aiweirdness.com/).
* Hayes, Brian. “Computer recreations.” Scientific American, vol. 249, no. 5, 1983, pp. 18–31. JSTOR, http://www.jstor.org/stable/24969024. (Original column from Scientific American that described how Markov chain text generation works—very readable! I can send a PDF, hit me up.)
* [A Travesty Generator for Micros](https://elmcip.net/critical-writing/travesty-generator-micros) is a follow-up to Hayes' article that has some more theory and an actual Pascal listing (which is now mostly of only historical interest).
* [This notebook](https://github.com/aparrish/rwet/blob/master/ngrams-and-markov-chains.ipynb) shows how to implement a Markov chain generator from scratch in Python, if you're interested in such things!